<a href="https://colab.research.google.com/github/devindatt/AIDD_HAR_Demo/blob/master/KumoCoderpadProb1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KUMO CODERPAD PROBLEM #1

**Problem #1:** Create training labels for the “user visit propensity” model.
Once the model is trained, for each user, the model should be able to
predict the probability the user will visit the website at least once in the
following 28 days.

In [25]:
#Necessary Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from datetime import timedelta  #to calculate difference in dates


Task 1: create training labels for the “user visit propensity” model.

Assumptions:
- The task only asked to identify session, nothing to do with what items were purchased.

In [26]:
df_users = pd.read_csv('/content/sample_data/ecommerce-users.csv')
df_sessions = pd.read_csv('/content/sample_data/ecommerce-sessions.csv')

In [27]:
df_users.head(1)

,"user_id,num_purchases"
0,"3af7c0950afafe8e3826b69311135333a81a3fc0,0.0"


In [28]:
df_sessions.head(1)

,"user_id,session_id,timestamp"
0,"f23b9d140ce618287f533deba2c6a194589999aa,c3f10..."


In [29]:
#Function to cleanup the dataset and column names
def make_col_list(df):
  col = []

  for i in df.columns:
    r = str(i).strip('[]').split(',')
    for j in r:
      col.append(j)

    df = df[i].str.split(',', expand=True)

  df.columns = col
  return df

In [30]:
df_users = make_col_list(df_users)
df_users.head()

,user_id,num_purchases
0,3af7c0950afafe8e3826b69311135333a81a3fc0,0.0
1,1207798b51a5f83f3f6ca3f6a7178f9ef7c7c534,0.0
2,039989f7f9e970c353cda19d4d32d39d16066630,0.0
3,d81aa88ac6a5e8c0218fcd88d1e5022a1388f210,0.0
4,8251d708599ed8fc4cc515b78e884f1df7c91f2d,0.0


In [31]:
df_sessions = make_col_list(df_sessions)
df_sessions.head()

,user_id,session_id,timestamp
0,f23b9d140ce618287f533deba2c6a194589999aa,c3f1024eddc377e0c965a9be51760b8c,2020-11-11 23:09:01 UTC
1,5edb3560b03fca82be4cb6f1549a57175f7bd211,5035d06c1852adfeff0a1dec01bc8dc8,2020-12-27 16:35:30 UTC
2,6f4495df83d93bad397b6e1719f9f9945aa0ba3e,64fe13aff1f8b1a3e2ccaa04a9cc8901,2020-10-29 19:33:50 UTC
3,bc5a4246f25ff5f97e692964b37379a38b31fc5e,8d6126e4db54e4135693255070874e86,2021-02-15 10:44:13 UTC
4,3809f294a9bf7695aae5b918b16798e0f570eb18,93bad65c1dad6b7837a34b4413a6211a,2020-12-15 20:11:39 UTC


In [32]:
# convert the 'Date' column to datetime format
df_sessions['timestamp']= pd.to_datetime(df_sessions['timestamp'])
print(df_sessions.head())

df_sessions

                                    user_id                        session_id  \
0  f23b9d140ce618287f533deba2c6a194589999aa  c3f1024eddc377e0c965a9be51760b8c   
1  5edb3560b03fca82be4cb6f1549a57175f7bd211  5035d06c1852adfeff0a1dec01bc8dc8   
2  6f4495df83d93bad397b6e1719f9f9945aa0ba3e  64fe13aff1f8b1a3e2ccaa04a9cc8901   
3  bc5a4246f25ff5f97e692964b37379a38b31fc5e  8d6126e4db54e4135693255070874e86   
4  3809f294a9bf7695aae5b918b16798e0f570eb18  93bad65c1dad6b7837a34b4413a6211a   

                  timestamp  
0 2020-11-11 23:09:01+00:00  
1 2020-12-27 16:35:30+00:00  
2 2020-10-29 19:33:50+00:00  
3 2021-02-15 10:44:13+00:00  
4 2020-12-15 20:11:39+00:00  


,user_id,session_id,timestamp
0,f23b9d140ce618287f533deba2c6a194589999aa,c3f1024eddc377e0c965a9be51760b8c,2020-11-11 23:09:01+00:00
1,5edb3560b03fca82be4cb6f1549a57175f7bd211,5035d06c1852adfeff0a1dec01bc8dc8,2020-12-27 16:35:30+00:00
2,6f4495df83d93bad397b6e1719f9f9945aa0ba3e,64fe13aff1f8b1a3e2ccaa04a9cc8901,2020-10-29 19:33:50+00:00
3,bc5a4246f25ff5f97e692964b37379a38b31fc5e,8d6126e4db54e4135693255070874e86,2021-02-15 10:44:13+00:00
4,3809f294a9bf7695aae5b918b16798e0f570eb18,93bad65c1dad6b7837a34b4413a6211a,2020-12-15 20:11:39+00:00
...,...,...,...
24912,baabb419f3e2a960c860d9adaba7cd93e9c80752,d71c32fe652c6fb080d29541f8486085,2020-10-21 11:01:25+00:00
24913,81fb1d6d8e77e1b8c0c899d4df2e93ff46ad2643,88ea6dd89b5dcb12a2f4003da6683cf3,2021-02-12 19:51:22+00:00
24914,224e7386c65cfe0c63bd1346174a9614f8a4d6fe,ef8d3c01c6c56fdf575a9855e4ed87db,2021-02-15 09:03:11+00:00
24915,9596de545f3332037ab566483bda6b21aa6bd138,74711994655f93aaa654c084014f92de,2020-10-16 14:18:43+00:00


In [33]:
#Need to preprocess the timestamp to proper dates to be able to calculate
df_sessions['timestamp'] = pd.to_datetime(df_sessions['timestamp'])
print(df_sessions.head())

                                    user_id                        session_id  \
0  f23b9d140ce618287f533deba2c6a194589999aa  c3f1024eddc377e0c965a9be51760b8c   
1  5edb3560b03fca82be4cb6f1549a57175f7bd211  5035d06c1852adfeff0a1dec01bc8dc8   
2  6f4495df83d93bad397b6e1719f9f9945aa0ba3e  64fe13aff1f8b1a3e2ccaa04a9cc8901   
3  bc5a4246f25ff5f97e692964b37379a38b31fc5e  8d6126e4db54e4135693255070874e86   
4  3809f294a9bf7695aae5b918b16798e0f570eb18  93bad65c1dad6b7837a34b4413a6211a   

                  timestamp  
0 2020-11-11 23:09:01+00:00  
1 2020-12-27 16:35:30+00:00  
2 2020-10-29 19:33:50+00:00  
3 2021-02-15 10:44:13+00:00  
4 2020-12-15 20:11:39+00:00  


In [34]:
#Create a label column for users visited in specific past windows

#Create a function to create labels for sessions initiated within 28 days
def create_labels(sessions, window=28):

    #First need to consolidate user sessions
    labels = [] #create empty list for complying a label feature
    users = sessions['user_id'].unique() #get unique users from the df

    #iterate thru the unique users and pull out their sessions, first & last dates
    for user in users:
        user_sessions = sessions[sessions['user_id'] == user]
        first_date = user_sessions['timestamp'].min()
        last_date = user_sessions['timestamp'].max()

        #print(user, first_date,last_date)

        current_date = first_date #set a pointer to the earliest session

        #While there is 28 days within the current and last session dates
        #Capture all the sessions events within this timeframe in a list
        while current_date + timedelta(days=window) <= last_date:
            next_date = current_date + timedelta(days=window)
            session_in_window = user_sessions[
                (user_sessions['timestamp'] >= current_date) &
                (user_sessions['timestamp'] < next_date)
                ]
            #If one or more sessions found in list then label 1, else negative
            label = 1 if len(session_in_window) > 0 else 0

            #Create new dataframe for all these new features & label for ML
            labels.append({
                'user_id': user,
                'start_date': current_date,
                'end_date': next_date,
                'label': label
            })

            #Move the pointer up one window size, and continue until no more
            current_date = next_date

    return pd.DataFrame(labels)

# Create the training dataset with labels
df_labels = create_labels(df_sessions)

# Display the labels
print(df_labels.head())

                                    user_id                start_date  \
0  5edb3560b03fca82be4cb6f1549a57175f7bd211 2020-11-18 18:52:37+00:00   
1  5edb3560b03fca82be4cb6f1549a57175f7bd211 2020-12-16 18:52:37+00:00   
2  5edb3560b03fca82be4cb6f1549a57175f7bd211 2021-01-13 18:52:37+00:00   
3  bc5a4246f25ff5f97e692964b37379a38b31fc5e 2021-01-24 20:59:25+00:00   
4  7680319e4ada656cb76544b2b76980a99cb5f90a 2021-01-24 08:48:12+00:00   

                   end_date  label  
0 2020-12-16 18:52:37+00:00      1  
1 2021-01-13 18:52:37+00:00      1  
2 2021-02-10 18:52:37+00:00      1  
3 2021-02-21 20:59:25+00:00      1  
4 2021-02-21 08:48:12+00:00      1  


Task 2: Once the model is trained, for each user, the model should be able to predict the probability the user will visit the website at least once in the following 28 days.

Assumptions:
- The amount visits in a 28 day window in the past has the same probability to predict the future visit in 28 days if one or more occur.

In [35]:
#Merge the labels with the user to have one compiled dataset
data = pd.merge(df_labels, df_users, on='user_id')

In [36]:
#Drop unnecessary columns and separate out the Features from labels
X = data.drop(columns=['user_id', 'start_date', 'end_date', 'label'])
y = data['label']

In [37]:
#Split dataset into train, validation, and holdout datasets (40/20/20)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_holdout, y_val, y_holdout = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [38]:
#Train & fit a Random Forest Classifier on the users visit patterns from data
model = RandomForestClassifier(n_estimators=150, random_state=21)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=150, random_state=21)

In [39]:
#Validate the model
y_val_pred = model.predict(X_val)
y_val_prob = model.predict_proba(X_val)[:, 1]

#Evaluate the model
accuracy = accuracy_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, y_val_prob)

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation ROC AUC: {roc_auc:.4f}")

Validation Accuracy: 0.6374
Validation ROC AUC: 0.4309


In [40]:
#Make predictions on the holdout set
y_holdout_pred = model.predict(X_holdout)
y_holdout_prob = model.predict_proba(X_holdout)[:, 1]

#Evaluate the holdout performance
holdout_accuracy = accuracy_score(y_holdout, y_holdout_pred)
holdout_roc_auc = roc_auc_score(y_holdout, y_holdout_prob)

print(f"Holdout Accuracy: {holdout_accuracy:.4f}")
print(f"Holdout ROC AUC: {holdout_roc_auc:.4f}")

Holdout Accuracy: 0.6374
Holdout ROC AUC: 0.5220


In [41]:
#Save the train, validation, and holdout sets
train_df = pd.concat([X_train, y_train], axis=1)
validation_df = pd.concat([X_val, y_val], axis=1)
holdout_df = pd.concat([X_holdout, y_holdout], axis=1)

train_df.to_csv('train.csv', index=False)
validation_df.to_csv('validation.csv', index=False)
holdout_df.to_csv('holdout.csv', index=False)

print("Datasets saved as train.csv, validation.csv, and holdout.csv.")

Datasets saved as train.csv, validation.csv, and holdout.csv.
